<a href="https://colab.research.google.com/github/mars241/NLP_Natural-Language-Processing/blob/main/AAM_WCS__NLP_5_TF_IDF_Term_Frequency%E2%80%93Inverse_Document_Frequency.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 🦾Chalenge -Tweetons ! 
# Méthode du "TfIdf Vectorizer"
TF-IDF signifie term frequency–inverse document frequency pour explorer le sentiment analysis.

Nous allons effectuer les mêmes missions que dans la quête précédente, afin de comparer les deux méthodes (TfIdf Vectorizer et CountVectorizer). Pour rappel, les missions étaient les suivantes :

1. Importe cet ensemble de données de tweets dans un DataFrame.
2. Ne garde que les tweets positifs et négatifs (tu excluras donc les neutral). Quel est le pourcentage de tweets positifs/négatifs ?
3. Copie la colonne text dans une Série X, et la colonne sentiment dans une Série y. Applique un train test split avec le random_state = 32.
4. Crée un modèle vectorizer avec scikit-learn en utilisant la méthode TfidfVectorizer. Entraîne ton modèle sur X_train, puis crée une matrice de features X_train_CV. 
Crée la matrice X_test_CV sans ré-entraîner le modèle. Le format de la matrice X_test_CV doit être 4091x15806 avec 44633 stored elements.
5. Entraîne maintenant une régression logistique avec les paramètres par défaut. Tu devrais obtenir les résultats suivants : 0.932 pour le test d'entraînement, et 0.873 pour l'ensemble de test.
6. Étape bonus : essaye d'afficher 10 tweets qui ont été mal prédits (faux positifs ou faux négatifs). Aurais-tu fait mieux que l'algorithme ?

# Critères de validation
Le notebook est consultable pour toute personne disposant du lien fourni par l'élève.

TfidfVectorizer est utilisée pour le preprocessing

La classification de tweets est faite grâce à un algorithme de régression logistique.

La performance de l'algorithme de régression logistique est mesurée.
Toutes les étapes sont réalisées.


1) Importe cet ensemble de données de tweets dans un DataFrame.

In [2]:
# Importation de la Bibliothèque pour la lecture/traitement de DF (train.csv)
import pandas as pd


In [3]:
# Librarie / Module importation fichier
from google.colab import files

# Création de la variable mommé ici "uploaded", qui contiendra ici votre fichier DF ("train.csv")pris dans un dossier ties et mise à la racine de google colab
uploaded = files.upload()

Saving train.csv to train.csv


In [4]:
# Lire et afficher le datafram depuis la racine google colab
import pandas as pd
df = pd.read_csv("train.csv")
df

,textID,text,selected_text,sentiment
0,cb774db0d1,"I`d have responded, if I were going","I`d have responded, if I were going",neutral
1,549e992a42,Sooo SAD I will miss you here in San Diego!!!,Sooo SAD,negative
2,088c60f138,my boss is bullying me...,bullying me,negative
3,9642c003ef,what interview! leave me alone,leave me alone,negative
4,358bd9e861,"Sons of ****, why couldn`t they put them on t...","Sons of ****,",negative
...,...,...,...,...
27475,4eac33d1c0,wish we could come see u on Denver husband l...,d lost,negative
27476,4f4c4fc327,I`ve wondered about rake to. The client has ...,", don`t force",negative
27477,f67aae2310,Yay good for both of you. Enjoy the break - y...,Yay good for both of you.,positive
27478,ed167662a5,But it was worth it ****.,But it was worth it ****.,positive


In [5]:
# plus d'info sur le contenu du DF
df.info()

# 4 colonnes & 27480 lignes

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 27480 entries, 0 to 27479
Data columns (total 4 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   textID         27480 non-null  object
 1   text           27480 non-null  object
 2   selected_text  27480 non-null  object
 3   sentiment      27480 non-null  object
dtypes: object(4)
memory usage: 858.9+ KB


In [8]:
# la colonne "sentiment" semble nous intéressé, vérifions les valeurs possibles qu'elle contient:
df['sentiment'].unique()

array(['neutral', 'negative', 'positive'], dtype=object)

2) Ne garde que les tweets positifs et négatifs (tu excluras donc les neutral). Quel est le pourcentage de tweets positifs/négatifs ?

In [7]:
# Calcule du pourcentage global des tweets positifs & négatifs sans traitement préalable des nan dans le DF
pourcentage = (1 - len(df[df['sentiment'] == 'neutral'])/len(df)) * 100
print('la part des tweets positifs & négatifs est de :', round(pourcentage, 2),'% tout parti confondu dans le DataFram') # arrondi au 2eme chiffre après la virgule

la part des tweets positifs & négatifs est de : 100.0 % tout parti confondu dans le DataFram


In [6]:
# Autrement et mieux:
# Exclusion des "neutral"(s) du DF:
df = df[df['sentiment']!='neutral']
# Calcule des pourcentages de tweets positifs/négatifs distinctement (Par la Méthode de "normalization" ) :
df['sentiment'].value_counts(normalize=True)*100

positive    52.447595
negative    47.552405
Name: sentiment, dtype: float64

3) Copie la colonne text dans une Série X, et la colonne sentiment dans une Série y. Applique un train test split avec le random_state = 32.

In [8]:
# importation du module pour le "train test split "
from sklearn.model_selection import train_test_split


X = df['text']
y = df['sentiment']

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=32)


4) Crée un modèle vectorizer avec scikit-learn en utilisant la méthode TfidfVectorizer. Entraîne ton modèle sur X_train, puis crée une matrice de features X_train_CV. Crée la matrice X_test_CV sans ré-entraîner le modèle. Le format de la matrice X_test_CV doit être 4091x15806 avec 44633 stored elements.

In [9]:
# importation du module pour le TF-IDF : Term Frequency–Inverse Document Frequency
from sklearn.feature_extraction.text import TfidfVectorizer

# utilisation de la méthode TfidfVectorizer (TF-IDF)
tfidf = TfidfVectorizer()

# Entraîner le modèle sur X_train
X_train_tfidf = tfidf.fit_transform(X_train) 
# ICI le nom de "la matrice de features < X_train_CV >" de la méthode " CountVectorizer" est bien re-mommé " < X_train_tfidf > " pour la méthode "TfidfVectorizer"

# Créer la matrice X_test_CV sans ré-entraînement/test du modèle
X_test_tfidf = tfidf.transform(X_test)


X_train_tfidf

# NB : la dimension de la matrice "X_train_tfidf" est de 12272 x 15806

<12272x15806 sparse matrix of type '<class 'numpy.float64'>'
	with 144578 stored elements in Compressed Sparse Row format>

In [10]:
# METRIQUES du ML
# Création d'un DF pour afficher les résultats du "train test split" : en particulier,avec la matrice de features "X_train_tfidf"

pd.DataFrame(X_train_tfidf.toarray(),columns = tfidf.get_feature_names_out())

,00,000,00am,01,024,02mxjj,03,04,05,06,...,½re,½rio,½s,½t,½tearï,½timo,½ve,½you,½z,½ï
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12267,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
12268,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
12269,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
12270,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [11]:
# ...Et pour le "Test"
X_test_tfidf = tfidf.transform(X_test)
X_test_tfidf

# NB: la dimension de la matrice "X_test_tfidf" est de 4091 x 15806.
# Est-ce bien le résultat attendue : " Le format de la matrice X_test_CV doit être 4091x15806 avec 44633 stored elements " ?
# Réponse : OUI; Bingo !!! 

<4091x15806 sparse matrix of type '<class 'numpy.float64'>'
	with 44633 stored elements in Compressed Sparse Row format>

In [12]:
# METRIQUES DU ML
# Stockons/Affichons cette denière (la matrice "X_test_tfidf") dans un DF:

pd.DataFrame(X_test_tfidf.toarray(),columns = tfidf.get_feature_names_out())

,00,000,00am,01,024,02mxjj,03,04,05,06,...,½re,½rio,½s,½t,½tearï,½timo,½ve,½you,½z,½ï
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4086,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4087,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4088,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4089,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


5) Entraîne maintenant une régression logistique avec les paramètres par défaut. Tu devrais obtenir les résultats suivants : 0.932 pour le test d'entraînement, et 0.873 pour l'ensemble de test.

In [13]:
# Régression logistique

# importation du module ML adéquate de SkLearn
from sklearn.linear_model import LogisticRegression


# Ici, nous entraînons le modèle uniquement sur l'ensemble de données d'entraînement.
model = LogisticRegression()

# Entraîner le modèle sur les données d'entraînement
model.fit(X_train_tfidf, y_train)


# Et maintenant nous comparons les deux scores (en arrondi de 3 chiffres après la virgule par le format '%.3f'%) :
print("\nLe Score d'exactitude (accuracy) sur l'ensemble d'entraînement :", '%.3f'%model.score(X_train_tfidf, y_train))
print("Le Score d'exactitude (accuracy) sur l'ensemble de test :", '%.3f'%model.score(X_test_tfidf, y_test))


Le Score d'exactitude (accuracy) sur l'ensemble d'entraînement : 0.932
Le Score d'exactitude (accuracy) sur l'ensemble de test : 0.873


In [14]:
# METRIQUES du ML
# Représentons la matrice de confusion de la régression logistique :

# importation du module pour la matrice de confusion
from sklearn.metrics import confusion_matrix

pd.DataFrame(data = confusion_matrix(y_true = y_test, y_pred = model.predict(X_test_tfidf)),
             index = model.classes_ + " Réels",
             columns = model.classes_ + " Prédites")

,negative Prédites,positive Prédites
negative Réels,1692,243
positive Réels,276,1880


6) Méthode1 :

Étape bonus : essaye d'afficher 10 tweets qui ont été mal prédits (faux positifs ou faux négatifs). Aurais-tu fait mieux que l'algorithme ?

In [15]:
# Afficher 10 tweets mals prédits (faux positifs ou faux négatifs).

predictions = model.predict(X_test_tfidf)

# Obtenir les indices des tweets incorrectement prédits
import numpy as np
y_testArray = y_test.to_numpy()
X_testArray = X_test.to_numpy()

incorrect_indices = [i for i in range(len(y_test)) if y_testArray[i] != predictions[i]]

# Imprimer les tweets qui ont été mal prédits
for i in incorrect_indices[:10]:
  print(f'Tweet: {X_testArray[i]}')
  print(f'Prediction: {predictions[i]}')
  print(f'Actual: {y_testArray[i]}')
  print()

Tweet: HollowbabesHere comes the utter shite #bgt <I completely agree
Prediction: positive
Actual: negative

Tweet:  SUFFICATION NO BREATHING. It`s okay. There`ll be more. You`re invited to mine, but I can`t promise fun times.  *Jinx
Prediction: positive
Actual: negative

Tweet: i wanna vote for Miley Cyrus for the mtv movie awards..but i don`t know where i could  somebody could send me a link? thaank you <3
Prediction: negative
Actual: positive

Tweet: I love music so much that i`ve gone through pain to play :S my sides of my fingers now are peeling and have blisters from playing so much
Prediction: positive
Actual: negative

Tweet: I can only message those who message me, if we`re fwends...so those that want replies..follow me.  hmm..that sounds funny..
Prediction: negative
Actual: positive

Tweet: wish I could feel no pain (8)  but it`s ok, at least they like Brazil!
Prediction: negative
Actual: positive

Tweet:  so glad i`m not at uni anymore
Prediction: negative
Actual: positive



6) Méthode 2:

Étape bonus : essaye d'afficher 10 tweets qui ont été mal prédits (faux positifs ou faux négatifs). Aurais-tu fait mieux que l'algorithme ?

In [20]:
# METRIQUES du ML :
# Autrement et mieux:
# implémentation des résultats dans le DF:

# importation du module pour le "train test split "
from sklearn.model_selection import train_test_split
# X,y et train_test_split
X = df.text
y = df.sentiment

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state = 32)


# importation du module pour le TF-IDF : Term Frequency–Inverse Document Frequency
from sklearn.feature_extraction.text import TfidfVectorizer

# Crée un modèle TF IDF
vectorizer = TfidfVectorizer()

# Entraîne ton modèle sur X_train
vectorizer.fit(X_train)

# créer sparse_matrix sur X_train
X_train_TFIDF = vectorizer.transform(X_train)

# créer sparse_matrix sur X_test SANS FIT
X_test_TFIDF = vectorizer.transform(X_test)

# format matrix de test
X_test_TFIDF

<4091x15806 sparse matrix of type '<class 'numpy.float64'>'
	with 44633 stored elements in Compressed Sparse Row format>

In [21]:
# Stocker les résultats prédits dans une colonne avec la méthode "vectorize"
X_pred = vectorizer.transform(X)

In [22]:
# Ajouter cette colonne prédite au DF
df['predicted'] = model.predict(X_pred)

In [23]:
# Afficher le DF initial mais aggrégé de la colonne prédite
df

,textID,text,selected_text,sentiment,predicted
1,549e992a42,Sooo SAD I will miss you here in San Diego!!!,Sooo SAD,negative,negative
2,088c60f138,my boss is bullying me...,bullying me,negative,negative
3,9642c003ef,what interview! leave me alone,leave me alone,negative,negative
4,358bd9e861,"Sons of ****, why couldn`t they put them on t...","Sons of ****,",negative,negative
6,6e0c6d75b1,2am feedings for the baby are fun when he is a...,fun,positive,positive
...,...,...,...,...,...
27474,b78ec00df5,enjoy ur night,enjoy,positive,positive
27475,4eac33d1c0,wish we could come see u on Denver husband l...,d lost,negative,negative
27476,4f4c4fc327,I`ve wondered about rake to. The client has ...,", don`t force",negative,negative
27477,f67aae2310,Yay good for both of you. Enjoy the break - y...,Yay good for both of you.,positive,positive


In [24]:
# Focus sur les 10 tweets mals prédits (faux positifs ou faux négatifs) dans le df :
df[df['sentiment'] != df['predicted']].head(10)

,textID,text,selected_text,sentiment,predicted
17,919fa93391,i`ve been sick for the past few days and thus...,sick,negative,positive
48,3d9d4b0b55,"i donbt like to peel prawns, i also dont like ...",dont like go,negative,positive
76,35f393a245,"WOW, i AM REALLY MiSSiN THE FAM(iLY) TODAY. BA...",BADDD.,negative,positive
81,58d382e07a,lucky kid...i so wanna see loserville pity i...,lucky,positive,negative
88,bffa3ddd61,i miss you bby wish you were going tomorr...,i miss you bby,negative,positive
103,fd9fb4f1ad,i realy wanted to go out cause its so nice but...,busy,negative,positive
125,6649f3558c,Not a prob hun,Not a prob,positive,negative
139,cb1b0d191d,"missed all the awesome weather, because she wa...","missed all the awesome weather,",negative,positive
141,997a62f83f,These kids are terrible! If I was in Good Evan...,terrible!,negative,positive
148,ae71041e21,Candle wax is very enjoyable.,enjoyable.,positive,negative
